In [ ]:
# nto25_notebook.ipynb

# %% [markdown]
# # NTO25 ML Competition Baseline
# Полный пайплайн для соревнования по машинному обучению

# %%
# Установка зависимостей (раскомментируйте при необходимости)
# !pip install lightgbm pandas numpy scikit-learn tqdm joblib pyarrow

# %%
import sys
import time
from pathlib import Path
from typing import Any

import joblib
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm

# %%
# =============================================================================
# КОНСТАНТЫ
# =============================================================================

# --- ФАЙЛЫ ---
# Используем абсолютные пути из оригинального кода
TRAIN_FILENAME = "/home/evstigneva/nto252/mix/train.csv"
TEST_FILENAME = "/home/evstigneva/nto252/test.csv" 
USER_DATA_FILENAME = "/home/evstigneva/nto252/mix/users.csv"
BOOK_DATA_FILENAME = "/home/evstigneva/nto252/mix/books.csv"
BOOK_GENRES_FILENAME = "/home/evstigneva/nto252/mix/book_genres.csv"
GENRES_FILENAME = "/home/evstigneva/nto252/mix/genres.csv"
BOOK_DESCRIPTIONS_FILENAME = "/home/evstigneva/nto252/mix/book_descriptions.csv"
SUBMISSION_FILENAME = "/home/evstigneva/nto252/submission30.11.25_7.csv"
TFIDF_VECTORIZER_FILENAME = "tfidf_vectorizer.pkl"
PROCESSED_DATA_FILENAME = "processed_features.parquet"

# --- НАЗВАНИЯ КОЛОНОК ---
COL_USER_ID = "user_id"
COL_BOOK_ID = "book_id"
COL_TARGET = "rating"
COL_SOURCE = "source"
COL_PREDICTION = "rating_predict"
COL_HAS_READ = "has_read"
COL_TIMESTAMP = "timestamp"

# Фичевые колонки (новые)
F_USER_MEAN_RATING = "user_mean_rating"
F_USER_RATINGS_COUNT = "user_ratings_count"
F_BOOK_MEAN_RATING = "book_mean_rating"
F_BOOK_RATINGS_COUNT = "book_ratings_count"
F_AUTHOR_MEAN_RATING = "author_mean_rating"
F_BOOK_GENRES_COUNT = "book_genres_count"

# Метаданные из сырых данных
COL_GENDER = "gender"
COL_AGE = "age"
COL_AUTHOR_ID = "author_id"
COL_PUBLICATION_YEAR = "publication_year"
COL_LANGUAGE = "language"
COL_PUBLISHER = "publisher"
COL_AVG_RATING = "avg_rating"
COL_GENRE_ID = "genre_id"
COL_DESCRIPTION = "description"

# --- ЗНАЧЕНИЯ ---
VAL_SOURCE_TRAIN = "train"
VAL_SOURCE_TEST = "test"

# --- МАГИЧЕСКИЕ ЧИСЛА ---
MISSING_CAT_VALUE = "-1"
MISSING_NUM_VALUE = -1
PREDICTION_MIN_VALUE = 0
PREDICTION_MAX_VALUE = 10

# %%
# =============================================================================
# КОНФИГУРАЦИЯ
# =============================================================================

# Определяем корневую директорию
ROOT_DIR = Path("/home/evstigneva/nto25/baseline").resolve()
DATA_DIR = ROOT_DIR / "data"
RAW_DATA_DIR = Path("/home/evstigneva/Zagr")  # Директория с исходными данными
INTERIM_DATA_DIR = DATA_DIR / "interim" 
PROCESSED_DATA_DIR = DATA_DIR / "processed"
OUTPUT_DIR = ROOT_DIR / "output"
MODEL_DIR = OUTPUT_DIR / "models"
SUBMISSION_DIR = OUTPUT_DIR / "submissions"

# Создаем необходимые директории
for dir_path in [DATA_DIR, PROCESSED_DATA_DIR, MODEL_DIR, SUBMISSION_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

print("✅ Директории созданы:")
print(f"   - Данные: {DATA_DIR}")
print(f"   - Модели: {MODEL_DIR}")
print(f"   - Результаты: {SUBMISSION_DIR}")
print(f"   - Исходные данные: {RAW_DATA_DIR}")

# Проверяем наличие файлов данных
print("\n🔍 Проверка файлов данных:")
data_files = [
    TRAIN_FILENAME, TEST_FILENAME, USER_DATA_FILENAME, 
    BOOK_DATA_FILENAME, BOOK_GENRES_FILENAME, GENRES_FILENAME, BOOK_DESCRIPTIONS_FILENAME
]

for file_path in data_files:
    if Path(file_path).exists():
        print(f"   ✅ {Path(file_path).name} - найден")
    else:
        print(f"   ❌ {Path(file_path).name} - НЕ НАЙДЕН")

# --- ПАРАМЕТРЫ ---
RANDOM_STATE = 42
TARGET = COL_TARGET

# --- КОНФИГУРАЦИЯ ВРЕМЕННОГО РАЗДЕЛЕНИЯ ---
TEMPORAL_SPLIT_RATIO = 0.8

# --- КОНФИГУРАЦИЯ ОБУЧЕНИЯ ---
EARLY_STOPPING_ROUNDS = 50
MODEL_FILENAME = "lgb_model.txt"

# --- ПАРАМЕТРЫ TF-IDF ---
TFIDF_MAX_FEATURES = 500
TFIDF_MIN_DF = 2
TFIDF_MAX_DF = 0.95
TFIDF_NGRAM_RANGE = (1, 2)

# --- ФИЧИ ---
CAT_FEATURES = [
    COL_USER_ID,
    COL_BOOK_ID,
    COL_GENDER,
    COL_AGE,
    COL_AUTHOR_ID,
    COL_PUBLICATION_YEAR,
    COL_LANGUAGE,
    COL_PUBLISHER,
]

# --- ПАРАМЕТРЫ МОДЕЛИ ---
LGB_PARAMS = {
    "objective": "rmse",
    "metric": "rmse",
    "n_estimators": 2000,
    "learning_rate": 0.01,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    "lambda_l1": 0.1,
    "lambda_l2": 0.1,
    "num_leaves": 31,
    "verbose": -1,
    "n_jobs": -1,
    "seed": RANDOM_STATE,
    "boosting_type": "gbdt",
}

LGB_FIT_PARAMS = {
    "eval_metric": "rmse",
    "callbacks": [],
}

# %%
# =============================================================================
# ФУНКЦИИ ОБРАБОТКИ ДАННЫХ
# =============================================================================

def load_and_merge_data():
    """Загружает сырые данные и объединяет их в единый DataFrame."""
    print("Загрузка данных...")

    # Определяем типы данных для оптимизации памяти
    dtype_spec = {
        COL_USER_ID: "int32",
        COL_BOOK_ID: "int32",
        COL_TARGET: "float32",
        COL_GENDER: "category",
        COL_AGE: "float32",
        COL_AUTHOR_ID: "int32",
        COL_PUBLICATION_YEAR: "float32",
        COL_LANGUAGE: "category",
        COL_PUBLISHER: "category",
        COL_AVG_RATING: "float32",
        COL_GENRE_ID: "int16",
    }

    # Загружаем датасеты
    train_df = pd.read_csv(
        TRAIN_FILENAME,
        dtype={
            k: v
            for k, v in dtype_spec.items()
            if k in [COL_USER_ID, COL_BOOK_ID, COL_TARGET]
        },
        parse_dates=[COL_TIMESTAMP],
    )

    # Фильтруем тренировочные данные: только книги с рейтингом (has_read=1)
    initial_count = len(train_df)
    train_df = train_df[train_df[COL_HAS_READ] == 1].copy()
    filtered_count = len(train_df)
    print(f"Отфильтровано тренировочных данных: {initial_count} -> {filtered_count} строк (только has_read=1)")
    
    test_df = pd.read_csv(
        TEST_FILENAME,
        dtype={k: v for k, v in dtype_spec.items() if k in [COL_USER_ID, COL_BOOK_ID]},
    )
    user_data_df = pd.read_csv(
        USER_DATA_FILENAME,
        dtype={
            k: v for k, v in dtype_spec.items() if k in [COL_USER_ID, COL_GENDER, COL_AGE]
        },
    )
    book_data_df = pd.read_csv(
        BOOK_DATA_FILENAME,
        dtype={
            k: v
            for k, v in dtype_spec.items()
            if k
            in [
                COL_BOOK_ID,
                COL_AUTHOR_ID,
                COL_PUBLICATION_YEAR,
                COL_LANGUAGE,
                COL_AVG_RATING,
                COL_PUBLISHER,
            ]
        },
    )
    book_genres_df = pd.read_csv(
        BOOK_GENRES_FILENAME,
        dtype={k: v for k, v in dtype_spec.items() if k in [COL_BOOK_ID, COL_GENRE_ID]},
    )
    genres_df = pd.read_csv(GENRES_FILENAME)
    book_descriptions_df = pd.read_csv(
        BOOK_DESCRIPTIONS_FILENAME,
        dtype={COL_BOOK_ID: "int32"},
    )

    print("Данные загружены. Объединение датасетов...")

    # Объединяем train и test
    train_df[COL_SOURCE] = VAL_SOURCE_TRAIN
    test_df[COL_SOURCE] = VAL_SOURCE_TEST
    combined_df = pd.concat([train_df, test_df], ignore_index=True, sort=False)

    # Добавляем метаданные
    combined_df = combined_df.merge(user_data_df, on=COL_USER_ID, how="left")

    # Удаляем дубликаты из book_data_df перед объединением
    book_data_df = book_data_df.drop_duplicates(subset=[COL_BOOK_ID])
    combined_df = combined_df.merge(book_data_df, on=COL_BOOK_ID, how="left")

    print(f"Размер объединенных данных: {combined_df.shape}")
    return combined_df, book_genres_df, genres_df, book_descriptions_df

# %%
# =============================================================================
# ФУНКЦИИ ФИЧЕЙ
# =============================================================================

def add_aggregate_features(df, train_df):
    """Вычисляет и добавляет агрегированные фичи пользователей, книг и авторов."""
    print("Добавление агрегированных фич...")

    # Агрегаты по пользователям
    user_agg = train_df.groupby(COL_USER_ID)[TARGET].agg(["mean", "count"]).reset_index()
    user_agg.columns = [
        COL_USER_ID,
        F_USER_MEAN_RATING,
        F_USER_RATINGS_COUNT,
    ]

    # Агрегаты по книгам
    book_agg = train_df.groupby(COL_BOOK_ID)[TARGET].agg(["mean", "count"]).reset_index()
    book_agg.columns = [
        COL_BOOK_ID,
        F_BOOK_MEAN_RATING,
        F_BOOK_RATINGS_COUNT,
    ]

    # Агрегаты по авторам
    author_agg = train_df.groupby(COL_AUTHOR_ID)[TARGET].agg(["mean"]).reset_index()
    author_agg.columns = [COL_AUTHOR_ID, F_AUTHOR_MEAN_RATING]

    # Объединяем агрегаты с основным датафреймом
    df = df.merge(user_agg, on=COL_USER_ID, how="left")
    df = df.merge(book_agg, on=COL_BOOK_ID, how="left")
    return df.merge(author_agg, on=COL_AUTHOR_ID, how="left")

def add_genre_features(df, book_genres_df):
    """Вычисляет и добавляет количество жанров для каждой книги."""
    print("Добавление фич жанров...")
    genre_counts = book_genres_df.groupby(COL_BOOK_ID)[COL_GENRE_ID].count().reset_index()
    genre_counts.columns = [
        COL_BOOK_ID,
        F_BOOK_GENRES_COUNT,
    ]
    return df.merge(genre_counts, on=COL_BOOK_ID, how="left")

def add_text_features(df, train_df, descriptions_df):
    """Добавляет TF-IDF фичи из описаний книг."""
    print("Добавление текстовых фич (TF-IDF)...")

    vectorizer_path = MODEL_DIR / TFIDF_VECTORIZER_FILENAME

    # Получаем уникальные книги из тренировочного набора
    train_books = train_df[COL_BOOK_ID].unique()

    # Извлекаем описания только для тренировочных книг
    train_descriptions = descriptions_df[descriptions_df[COL_BOOK_ID].isin(train_books)].copy()
    train_descriptions[COL_DESCRIPTION] = train_descriptions[COL_DESCRIPTION].fillna("")

    # Проверяем существование векторайзера (для предсказания)
    if vectorizer_path.exists():
        print(f"Загрузка существующего векторайзера из {vectorizer_path}")
        vectorizer = joblib.load(vectorizer_path)
    else:
        # Обучаем векторайзер только на тренировочных описаниях
        print("Обучение TF-IDF векторайзера на тренировочных описаниях...")
        vectorizer = TfidfVectorizer(
            max_features=TFIDF_MAX_FEATURES,
            min_df=TFIDF_MIN_DF,
            max_df=TFIDF_MAX_DF,
            ngram_range=TFIDF_NGRAM_RANGE,
        )
        vectorizer.fit(train_descriptions[COL_DESCRIPTION])
        # Сохраняем векторайзер для использования в предсказании
        joblib.dump(vectorizer, vectorizer_path)
        print(f"Векторайзер сохранен в {vectorizer_path}")

    # Трансформируем все описания книг
    all_descriptions = descriptions_df[[COL_BOOK_ID, COL_DESCRIPTION]].copy()
    all_descriptions[COL_DESCRIPTION] = all_descriptions[COL_DESCRIPTION].fillna("")

    # Создаем маппинг book_id -> description
    description_map = dict(
        zip(all_descriptions[COL_BOOK_ID], all_descriptions[COL_DESCRIPTION], strict=False)
    )

    # Получаем описания для книг в df (в том же порядке)
    df_descriptions = df[COL_BOOK_ID].map(description_map).fillna("")

    # Трансформируем в TF-IDF фичи
    tfidf_matrix = vectorizer.transform(df_descriptions)

    # Конвертируем разреженную матрицу в DataFrame
    tfidf_feature_names = [f"tfidf_{i}" for i in range(tfidf_matrix.shape[1])]
    tfidf_df = pd.DataFrame(
        tfidf_matrix.toarray(),
        columns=tfidf_feature_names,
        index=df.index,
    )

    # Объединяем TF-IDF фичи с основным DataFrame
    df_with_tfidf = pd.concat([df.reset_index(drop=True), tfidf_df.reset_index(drop=True)], axis=1)

    print(f"Добавлено {len(tfidf_feature_names)} TF-IDF фич.")
    return df_with_tfidf

def handle_missing_values(df, train_df):
    """Заполняет пропущенные значения используя определенную стратегию."""
    print("Обработка пропущенных значений...")

    # Вычисляем глобальное среднее из тренировочных данных для заполнения
    global_mean = train_df[TARGET].mean()

    # Заполняем возраст медианой
    age_median = df[COL_AGE].median()
    df[COL_AGE] = df[COL_AGE].fillna(age_median)

    # Заполняем агрегированные фичи для "холодного старта" пользователей/предметов
    if F_USER_MEAN_RATING in df.columns:
        df[F_USER_MEAN_RATING] = df[F_USER_MEAN_RATING].fillna(global_mean)
    if F_BOOK_MEAN_RATING in df.columns:
        df[F_BOOK_MEAN_RATING] = df[F_BOOK_MEAN_RATING].fillna(global_mean)
    if F_AUTHOR_MEAN_RATING in df.columns:
        df[F_AUTHOR_MEAN_RATING] = df[F_AUTHOR_MEAN_RATING].fillna(global_mean)

    if F_USER_RATINGS_COUNT in df.columns:
        df[F_USER_RATINGS_COUNT] = df[F_USER_RATINGS_COUNT].fillna(0)
    if F_BOOK_RATINGS_COUNT in df.columns:
        df[F_BOOK_RATINGS_COUNT] = df[F_BOOK_RATINGS_COUNT].fillna(0)

    # Заполняем avg_rating из book_data глобальным средним
    df[COL_AVG_RATING] = df[COL_AVG_RATING].fillna(global_mean)

    # Заполняем счетчики жанров нулями
    df[F_BOOK_GENRES_COUNT] = df[F_BOOK_GENRES_COUNT].fillna(0)

    # Заполняем TF-IDF фичи нулями (для книг без описаний)
    tfidf_cols = [col for col in df.columns if col.startswith("tfidf_")]
    for col in tfidf_cols:
        df[col] = df[col].fillna(0.0)

    # Заполняем оставшиеся категориальные фичи специальным значением
    for col in CAT_FEATURES:
        if col in df.columns:
            if df[col].dtype.name in ("category", "object") and df[col].isna().any():
                df[col] = df[col].astype(str).fillna(MISSING_CAT_VALUE).astype("category")
            elif pd.api.types.is_numeric_dtype(df[col].dtype) and df[col].isna().any():
                df[col] = df[col].fillna(MISSING_NUM_VALUE)

    return df

def create_features(df, book_genres_df, descriptions_df, include_aggregates=False):
    """Запускает полный пайплайн инженерии фич."""
    print("Запуск пайплайна инженерии фич...")
    train_df = df[df[COL_SOURCE] == VAL_SOURCE_TRAIN].copy()

    # Агрегированные фичи вычисляются отдельно во время обучения
    if include_aggregates:
        df = add_aggregate_features(df, train_df)

    df = add_genre_features(df, book_genres_df)
    df = add_text_features(df, train_df, descriptions_df)
    df = handle_missing_values(df, train_df)

    # Конвертируем категориальные колонки в pandas 'category' dtype для LightGBM
    for col in CAT_FEATURES:
        if col in df.columns:
            df[col] = df[col].astype("category")

    print("Инженерия фич завершена.")
    return df

# %%
# =============================================================================
# ФУНКЦИИ ВРЕМЕННОГО РАЗДЕЛЕНИЯ
# =============================================================================

def temporal_split_by_date(df, split_date, timestamp_col=COL_TIMESTAMP):
    """Разделяет DataFrame на тренировочную и валидационную выборки по абсолютной дате."""
    if timestamp_col not in df.columns:
        raise ValueError(
            f"Колонка с временными метками '{timestamp_col}' не найдена в DataFrame. Доступные колонки: {df.columns.tolist()}"
        )

    # Убеждаемся, что временная метка в datetime формате
    if not pd.api.types.is_datetime64_any_dtype(df[timestamp_col]):
        df = df.copy()
        df[timestamp_col] = pd.to_datetime(df[timestamp_col])

    # Разделяем по порогу даты
    train_mask = df[timestamp_col] <= split_date
    val_mask = df[timestamp_col] > split_date

    # Валидационные проверки
    if train_mask.sum() == 0:
        raise ValueError(f"Не найдено записей с временной меткой <= {split_date}. Проверьте split_date.")

    if val_mask.sum() == 0:
        raise ValueError(f"Не найдено записей с временной меткой > {split_date}. Проверьте split_date.")

    # Дополнительная проверка безопасности: убеждаемся, что все валидационные временные метки после тренировочных
    if train_mask.sum() > 0 and val_mask.sum() > 0:
        max_train_timestamp = df.loc[train_mask, timestamp_col].max()
        min_val_timestamp = df.loc[val_mask, timestamp_col].min()

        if min_val_timestamp <= max_train_timestamp:
            raise ValueError(
                f"Валидация временного разделения не удалась: минимальная валидационная временная метка ({min_val_timestamp}) "
                f"не больше максимальной тренировочной временной метки ({max_train_timestamp}). "
                "Это указывает на проблему целостности данных."
            )

    return train_mask, val_mask

def get_split_date_from_ratio(df, ratio, timestamp_col=COL_TIMESTAMP):
    """Вычисляет дату разделения на основе соотношения точек данных."""
    if not 0 < ratio < 1:
        raise ValueError(f"Соотношение должно быть между 0 и 1, получено {ratio}")

    if timestamp_col not in df.columns:
        raise ValueError(f"Колонка с временными метками '{timestamp_col}' не найдена в DataFrame.")

    # Убеждаемся, что временная метка в datetime формате
    if not pd.api.types.is_datetime64_any_dtype(df[timestamp_col]):
        df = df.copy()
        df[timestamp_col] = pd.to_datetime(df[timestamp_col])

    # Вычисляем порог на основе соотношения
    sorted_timestamps = df[timestamp_col].sort_values()
    threshold_index = int(len(sorted_timestamps) * ratio)

    return sorted_timestamps.iloc[threshold_index]

# %%
# =============================================================================
# ОСНОВНЫЕ ФУНКЦИИ ПАЙПЛАЙНА
# =============================================================================

def prepare_data():
    """Обрабатывает сырые данные и сохраняет подготовленные фичи в processed директорию."""
    print("=" * 60)
    print("Пайплайн подготовки данных")
    print("=" * 60)

    # Проверяем наличие необходимых файлов
    required_files = [TRAIN_FILENAME, TEST_FILENAME, USER_DATA_FILENAME, BOOK_DATA_FILENAME, 
                     BOOK_GENRES_FILENAME, GENRES_FILENAME, BOOK_DESCRIPTIONS_FILENAME]
    
    missing_files = [f for f in required_files if not Path(f).exists()]
    if missing_files:
        raise FileNotFoundError(f"Отсутствуют необходимые файлы: {missing_files}")

    # Загружаем и объединяем сырые данные
    merged_df, book_genres_df, _, descriptions_df = load_and_merge_data()

    # Применяем инженерию фич БЕЗ агрегатов
    featured_df = create_features(merged_df, book_genres_df, descriptions_df, include_aggregates=False)

    # Определяем путь вывода
    processed_path = PROCESSED_DATA_DIR / PROCESSED_DATA_FILENAME

    # Сохраняем обработанные данные как parquet для эффективности
    print(f"\nСохранение обработанных данных в {processed_path}...")
    featured_df.to_parquet(processed_path, index=False, engine="pyarrow", compression="snappy")
    print("Обработанные данные успешно сохранены!")

    # Печатаем статистику
    train_rows = len(featured_df[featured_df[COL_SOURCE] == VAL_SOURCE_TRAIN])
    test_rows = len(featured_df[featured_df[COL_SOURCE] == VAL_SOURCE_TEST])
    total_features = len(featured_df.columns)

    print("\nПодготовка данных завершена!")
    print(f"  - Тренировочные строки: {train_rows:,}")
    print(f"  - Тестовые строки: {test_rows:,}")
    print(f"  - Всего фич: {total_features}")
    print(f"  - Выходной файл: {processed_path}")
    
    return featured_df

def train():
    """Запускает пайплайн обучения модели с временным разделением."""
    # Загружаем подготовленные данные
    processed_path = PROCESSED_DATA_DIR / PROCESSED_DATA_FILENAME

    if not processed_path.exists():
        raise FileNotFoundError(
            f"Обработанные данные не найдены в {processed_path}. "
            "Пожалуйста, сначала запустите prepare_data()."
        )

    print(f"Загрузка подготовленных данных из {processed_path}...")
    featured_df = pd.read_parquet(processed_path, engine="pyarrow")
    print(f"Загружено {len(featured_df):,} строк с {len(featured_df.columns)} фичами")

    # Разделяем тренировочные и тестовые наборы
    train_set = featured_df[featured_df[COL_SOURCE] == VAL_SOURCE_TRAIN].copy()

    # Проверяем наличие колонки временных меток
    if COL_TIMESTAMP not in train_set.columns:
        raise ValueError(
            f"Колонка с временными метками '{COL_TIMESTAMP}' не найдена в тренировочном наборе. "
            "Убедитесь, что данные были подготовлены с сохранением временных меток."
        )

    # Убеждаемся, что временная метка в datetime формате
    if not pd.api.types.is_datetime64_any_dtype(train_set[COL_TIMESTAMP]):
        train_set[COL_TIMESTAMP] = pd.to_datetime(train_set[COL_TIMESTAMP])

    # Выполняем временное разделение
    print(f"\nВыполнение временного разделения с соотношением {TEMPORAL_SPLIT_RATIO}...")
    split_date = get_split_date_from_ratio(train_set, TEMPORAL_SPLIT_RATIO, COL_TIMESTAMP)
    print(f"Дата разделения: {split_date}")

    train_mask, val_mask = temporal_split_by_date(train_set, split_date, COL_TIMESTAMP)

    # Разделяем данные
    train_split = train_set[train_mask].copy()
    val_split = train_set[val_mask].copy()

    print(f"Тренировочное разделение: {len(train_split):,} строк")
    print(f"Валидационное разделение: {len(val_split):,} строк")

    # Проверяем временную корректность
    max_train_timestamp = train_split[COL_TIMESTAMP].max()
    min_val_timestamp = val_split[COL_TIMESTAMP].min()
    print(f"Максимальная тренировочная временная метка: {max_train_timestamp}")
    print(f"Минимальная валидационная временная метка: {min_val_timestamp}")

    if min_val_timestamp <= max_train_timestamp:
        raise ValueError(
            f"Валидация временного разделения не удалась: минимальная валидационная временная метка ({min_val_timestamp}) "
            f"не больше максимальной тренировочной временной метки ({max_train_timestamp})."
        )
    print("✅ Валидация временного разделения пройдена: все валидационные временные метки после тренировочных")

    # Вычисляем агрегированные фичи только на тренировочном разделении (для предотвращения утечки данных)
    print("\nВычисление агрегированных фич только на тренировочном разделении...")
    train_split_with_agg = add_aggregate_features(train_split.copy(), train_split)
    val_split_with_agg = add_aggregate_features(val_split.copy(), train_split)  # Используем train_split для агрегатов!

    # Обрабатываем пропущенные значения (используем train_split для заполнения)
    print("Обработка пропущенных значений...")
    train_split_final = handle_missing_values(train_split_with_agg, train_split)
    val_split_final = handle_missing_values(val_split_with_agg, train_split)

    # Определяем фичи (X) и таргет (y)
    exclude_cols = [
        COL_SOURCE,
        TARGET,
        COL_PREDICTION,
        COL_TIMESTAMP,
    ]
    features = [col for col in train_split_final.columns if col not in exclude_cols]

    # Исключаем оставшиеся object колонки, которые не являются фичами модели
    non_feature_object_cols = train_split_final[features].select_dtypes(include=["object"]).columns.tolist()
    features = [f for f in features if f not in non_feature_object_cols]

    X_train = train_split_final[features]
    y_train = train_split_final[TARGET]
    X_val = val_split_final[features]
    y_val = val_split_final[TARGET]

    print(f"Тренировочные фичи: {len(features)}")

    # Обучаем единую модель
    print("\nОбучение LightGBM модели...")
    model = lgb.LGBMRegressor(**LGB_PARAMS)

    # Обновляем параметры fit с колбэком ранней остановки
    fit_params = LGB_FIT_PARAMS.copy()
    fit_params["callbacks"] = [lgb.early_stopping(stopping_rounds=EARLY_STOPPING_ROUNDS)]

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],
        eval_metric=fit_params["eval_metric"],
        callbacks=fit_params["callbacks"],
    )

    # Оцениваем модель
    val_preds = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, val_preds))
    mae = mean_absolute_error(y_val, val_preds)
    print(f"\nВалидационный RMSE: {rmse:.4f}, MAE: {mae:.4f}")

    # Сохраняем обученную модель
    model_path = MODEL_DIR / MODEL_FILENAME
    model.booster_.save_model(str(model_path))
    print(f"Модель сохранена в {model_path}")

    print("\nОбучение завершено.")
    
    return model, rmse, mae

def predict():
    """Генерирует и сохраняет предсказания для тестового набора."""
    # Загружаем подготовленные данные
    processed_path = PROCESSED_DATA_DIR / PROCESSED_DATA_FILENAME

    if not processed_path.exists():
        raise FileNotFoundError(
            f"Обработанные данные не найдены в {processed_path}. "
            "Пожалуйста, сначала запустите prepare_data()."
        )

    print(f"Загрузка подготовленных данных из {processed_path}...")
    featured_df = pd.read_parquet(processed_path, engine="pyarrow")
    print(f"Загружено {len(featured_df):,} строк с {len(featured_df.columns)} фичами")

    # Разделяем тренировочные и тестовые наборы
    train_set = featured_df[featured_df[COL_SOURCE] == VAL_SOURCE_TRAIN].copy()
    test_set = featured_df[featured_df[COL_SOURCE] == VAL_SOURCE_TEST].copy()

    print(f"Тренировочный набор: {len(train_set):,} строк")
    print(f"Тестовый набор: {len(test_set):,} строк")

    # Вычисляем агрегированные фичи на ВСЕХ тренировочных данных (для использования в тестовых предсказаниях)
    print("\nВычисление агрегированных фич на всех тренировочных данных...")
    test_set_with_agg = add_aggregate_features(test_set.copy(), train_set)

    # Обрабатываем пропущенные значения (используем train_set для заполнения)
    print("Обработка пропущенных значения...")
    test_set_final = handle_missing_values(test_set_with_agg, train_set)

    # Определяем фичи (исключаем source, target, prediction, timestamp колонки)
    exclude_cols = [
        COL_SOURCE,
        TARGET,
        COL_PREDICTION,
        COL_TIMESTAMP,
    ]
    features = [col for col in test_set_final.columns if col not in exclude_cols]

    # Исключаем оставшиеся object колонки, которые не являются фичами модели
    non_feature_object_cols = test_set_final[features].select_dtypes(include=["object"]).columns.tolist()
    features = [f for f in features if f not in non_feature_object_cols]

    X_test = test_set_final[features]
    print(f"Фичи для предсказания: {len(features)}")

    # Загружаем обученную модель
    model_path = MODEL_DIR / MODEL_FILENAME
    if not model_path.exists():
        raise FileNotFoundError(
            f"Модель не найдена в {model_path}. " 
            "Пожалуйста, сначала запустите train()."
        )

    print(f"\nЗагрузка модели из {model_path}...")
    model = lgb.Booster(model_file=str(model_path))

    # Генерируем предсказания
    print("Генерация предсказаний...")
    test_preds = model.predict(X_test)

    # Обрезаем предсказания до валидного диапазона рейтинга [0, 10]
    clipped_preds = np.clip(test_preds, PREDICTION_MIN_VALUE, PREDICTION_MAX_VALUE)

    # Создаем файл submission
    submission_df = test_set[[COL_USER_ID, COL_BOOK_ID]].copy()
    submission_df[COL_PREDICTION] = clipped_preds

    submission_path = SUBMISSION_DIR / SUBMISSION_FILENAME

    submission_df.to_csv(submission_path, index=False)
    print(f"\nФайл submission создан в: {submission_path}")
    print(f"Предсказания: min={clipped_preds.min():.4f}, max={clipped_preds.max():.4f}, mean={clipped_preds.mean():.4f}")
    
    return submission_df, clipped_preds

def validate():
    """Валидирует структуру и формат файла submission."""
    print("Валидация файла submission...")

    try:
        # Загружаем тестовые данные и файл submission
        test_df = pd.read_csv(TEST_FILENAME)
        sub_df = pd.read_csv(SUBMISSION_DIR / SUBMISSION_FILENAME)

        # 1. Проверяем длину
        assert len(sub_df) == len(test_df), f"Несоответствие длины submission. Ожидалось {len(test_df)}, получено {len(sub_df)}."
        print("✅ Проверка длины пройдена.")

        # 2. Проверяем пропущенные значения в предсказаниях
        assert (
            not sub_df[COL_PREDICTION].isna().any()
        ), f"Найдены пропущенные значения в '{COL_PREDICTION}'."
        print("✅ Проверка на отсутствие пропущенных значений пройдена.")

        # 3. Проверяем, что набор пар (user_id, book_id) совпадает
        test_keys = (
            test_df[[COL_USER_ID, COL_BOOK_ID]]
            .copy()
            .set_index([COL_USER_ID, COL_BOOK_ID])
        )
        sub_keys = (
            sub_df[[COL_USER_ID, COL_BOOK_ID]]
            .copy()
            .set_index([COL_USER_ID, COL_BOOK_ID])
        )

        assert test_keys.index.equals(
            sub_keys.index
        ), "Набор пар (user_id, book_id) не совпадает с тестовым набором."
        print("✅ Проверка совпадения пар (user_id, book_id) пройдена.")

        # 4. Проверяем диапазон предсказаний
        assert (
            sub_df[COL_PREDICTION]
            .between(PREDICTION_MIN_VALUE, PREDICTION_MAX_VALUE)
            .all()
        ), f"Предсказания не в диапазоне [{PREDICTION_MIN_VALUE}, {PREDICTION_MAX_VALUE}]."
        print("✅ Проверка диапазона предсказаний [0, 10] пройдена.")

        print("\nВалидация успешна! Файл submission имеет корректный формат.")
        return True

    except FileNotFoundError as e:
        print(f"Ошибка: {e}. Убедитесь, что необходимые файлы существуют.")
        return False
    except AssertionError as e:
        print(f"Валидация не удалась: {e}")
        return False
    except Exception as e:
        print(f"Произошла непредвиденная ошибка: {e}")
        return False

# %%
# =============================================================================
# ЗАПУСК ПАЙПЛАЙНА - ВЫБЕРИТЕ НУЖНЫЕ ЯЧЕЙКИ ДЛЯ ЗАПУСКА
# =============================================================================

# %% [markdown]
# ## 🚀 Запуск полного пайплайна
# Выполните ячейки ниже по порядку:

# %%
# 1. ПОДГОТОВКА ДАННЫХ
print("🎯 ШАГ 1: ПОДГОТОВКА ДАННЫХ")
featured_df = prepare_data()

# %%
# 2. ОБУЧЕНИЕ МОДЕЛИ
print("🎯 ШАГ 2: ОБУЧЕНИЕ МОДЕЛИ")
model, rmse, mae = train()

# %%
# 3. ПРЕДСКАЗАНИЕ
print("🎯 ШАГ 3: ПРЕДСКАЗАНИЕ")
submission_df, predictions = predict()

# %%
# 4. ВАЛИДАЦИЯ
print("🎯 ШАГ 4: ВАЛИДАЦИЯ")
validation_success = validate()

# %%
# 5. РЕЗУЛЬТАТЫ
print("🎯 ШАГ 5: РЕЗУЛЬТАТЫ")
print("\n" + "="*50)
print("📊 ИТОГОВЫЕ РЕЗУЛЬТАТЫ")
print("="*50)
print(f"✅ Подготовка данных: {len(featured_df):,} строк")
print(f"✅ Обучение модели: RMSE = {rmse:.4f}, MAE = {mae:.4f}")
print(f"✅ Предсказания: {len(predictions):,} прогнозов")
print(f"✅ Валидация: {'ПРОЙДЕНА' if validation_success else 'НЕ ПРОЙДЕНА'}")
print(f"📁 Файл submission: {SUBMISSION_DIR / SUBMISSION_FILENAME}")

# %% [markdown]
# ## 🔧 Запуск отдельных компонентов
# Если нужно запустить только определенные части:

# %%
# ТОЛЬКО ПОДГОТОВКА ДАННЫХ
# featured_df = prepare_data()

# %%
# ТОЛЬКО ОБУЧЕНИЕ
# model, rmse, mae = train()

# %%
# ТОЛЬКО ПРЕДСКАЗАНИЕ
# submission_df, predictions = predict()

# %%
# ТОЛЬКО ВАЛИДАЦИЯ
# validate()

# %% [markdown]
# ## 📊 Анализ данных (опционально)

# %%
# ПРОСМОТР ПОДГОТОВЛЕННЫХ ДАННЫХ
if 'featured_df' in locals():
    print("📈 Статистика подготовленных данных:")
    print(f"Размер: {featured_df.shape}")
    print(f"Колонки: {list(featured_df.columns)}")
    print("\nПервые 5 строк:")
    display(featured_df.head())

# %%
# СТАТИСТИКА ПРЕДСКАЗАНИЙ
if 'predictions' in locals():
    print("📊 Статистика предсказаний:")
    print(f"Min: {predictions.min():.4f}")
    print(f"Max: {predictions.max():.4f}") 
    print(f"Mean: {predictions.mean():.4f}")
    print(f"Std: {predictions.std():.4f}")
    
    # Гистограмма предсказаний
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 6))
    plt.hist(predictions, bins=50, alpha=0.7, color='skyblue')
    plt.title('Распределение предсказаний рейтингов')
    plt.xlabel('Рейтинг')
    plt.ylabel('Частота')
    plt.grid(True, alpha=0.3)
    plt.show()

In [5]:
import pandas as pd
import numpy as np

def process_ratings(base_file, update_file, output_file):
    """
    Обрабатывает рейтинги из двух файлов по заданным правилам.
    
    Args:
        base_file: путь к первому CSV файлу с исходными данными
        update_file: путь ко второму CSV файлу с предсказаниями
        output_file: путь для сохранения результата
    """
    
    # 1. Открываем первый CSV файл
    print("Загрузка первого файла...")
    try:
        df_base = pd.read_csv(base_file)
        print(f"Загружено строк из первого файла: {len(df_base)}")
    except Exception as e:
        print(f"Ошибка при загрузке первого файла: {e}")
        return
    
    # 2. Открываем второй CSV файл
    print("Загрузка второго файла...")
    try:
        df_update = pd.read_csv(update_file)
        print(f"Загружено строк из второго файла: {len(df_update)}")
    except Exception as e:
        print(f"Ошибка при загрузке второго файла: {e}")
        return
    
    # Проверяем наличие необходимых столбцов
    required_base_cols = ['user_id', 'book_id', 'rating_predict']
    required_update_cols = ['user_id', 'book_id', 'rating_predict']
    
    if not all(col in df_base.columns for col in required_base_cols):
        print(f"Первый файл должен содержать столбцы: {required_base_cols}")
        return
    
    if not all(col in df_update.columns for col in required_update_cols):
        print(f"Второй файл должен содержать столбцы: {required_update_cols}")
        return
    
    # Создаем словарь для быстрого поиска предсказаний
    print("Создание словаря предсказаний...")
    update_dict = {}
    for _, row in df_update.iterrows():
        key = (row['user_id'], row['book_id'])
        update_dict[key] = row['rating_predict']
    
    # 3-4. Обрабатываем строки
    print("Обработка рейтингов...")
    
    processed_count = 0
    updated_count = 0
    
    for idx, row in df_base.iterrows():
        key = (row['user_id'], row['book_id'])
        original_rating = row['rating_predict']
        
        if key in update_dict:
            rating_predict = update_dict[key]
            
            # 3. Если рейтинг от 0 до 2, заменяем на среднее + rating_predict
            if 0 <= original_rating <= 2:
                new_rating = (original_rating + rating_predict) / 2
                df_base.at[idx, 'rating_predict'] = new_rating
                processed_count += 1
            
            # 4. Для остальных значений изменяем на дельту
            else:
                delta = rating_predict - original_rating
                new_rating = original_rating + delta
                df_base.at[idx, 'rating_predict'] = new_rating
                updated_count += 1
    
    print(f"Обработано значений 0-2: {processed_count}")
    print(f"Обновлено других значений: {updated_count}")
    
    # 5. Округляем значения рейтинга
    print("Округление рейтингов...")
    df_base['rating_predict'] = df_base['rating_predict'].apply(lambda x: round(x) if not pd.isna(x) else x)
    
    # Проверяем, что рейтинги в допустимом диапазоне (например, 1-10)
    print("Проверка диапазона рейтингов...")
    df_base['rating_predict'] = df_base['rating_predict'].clip(lower=0, upper=10)  # Установите нужный диапазон
    
    df_base['rating_predict'] = df_base['rating_predict'].replace(9, 10)

    # 6. Сохраняем результат в новый CSV файл
    print(f"Сохранение результата в {output_file}...")
    try:
        df_base.to_csv(output_file, index=False)
        print(f"Файл успешно сохранен. Всего строк: {len(df_base)}")
        
        # Выводим статистику
        print("\nСтатистика по рейтингам:")
        print(f"Минимальный рейтинг: {df_base['rating_predict'].min()}")
        print(f"Максимальный рейтинг: {df_base['rating_predict'].max()}")
        print(f"Средний рейтинг: {df_base['rating_predict'].mean():.2f}")
        print(f"Медианный рейтинг: {df_base['rating_predict'].median()}")
        
        # Пример первых 5 строк
        print("\nПервые 5 строк результата:")
        print(df_base.head())
        
    except Exception as e:
        print(f"Ошибка при сохранении файла: {e}")
        return

# Пример использования
if __name__ == "__main__":
    # Укажите пути к вашим файлам
    base_file = "/home/evstigneva/Zagr/sample_submission.csv"  # Первый CSV файл
    update_file = "/home/evstigneva/Zagr/submission30.11.25_7.csv"  # Второй CSV файл с предсказаниями
    output_file = "/home/evstigneva/nto252/submission2.12.25_1.csv"  # Выходной файл
    
    # Запускаем обработку
    process_ratings(base_file, update_file, output_file)

Загрузка первого файла...
Загружено строк из первого файла: 2894
Загрузка второго файла...
Загружено строк из второго файла: 2894
Создание словаря предсказаний...
Обработка рейтингов...
Обработано значений 0-2: 0
Обновлено других значений: 2894
Округление рейтингов...
Проверка диапазона рейтингов...
Сохранение результата в /home/evstigneva/nto252/submission2.12.25_1.csv...
Файл успешно сохранен. Всего строк: 2894

Статистика по рейтингам:
Минимальный рейтинг: 2
Максимальный рейтинг: 10
Средний рейтинг: 7.82
Медианный рейтинг: 8.0

Первые 5 строк результата:
   user_id  book_id  rating_predict
0      281  2461928               8
1     1250    31957               7
2     4241   196603               8
3     5140   468894              10
4     7781  2141951               7
